In [2]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image
import shift_methods as sm
# from immethods import *

In [3]:
def mediancombine(filelist):
    '''
    median combine
    =======================
    Takes in a list of frames and does a medain combination on them.
    Used for biases, master flats, and basic image combination.
    
    inputs
    =======================
    filelist : The list of files you want to median combine
    
    returns
    =======================
    med_frame: The median combined frame
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame

In [8]:
os.chdir('E:/')
nights = glob.glob('2019*')
nights.sort()

In [9]:
for i in range(len(nights)):
    biasdir = 'E:/'+nights[i]+'/'
    os.chdir(biasdir)
    allims = glob.glob('c*.fits')
    biases = []
    try:
        for im in allims:
            if fits.getheader(im)['EXPTIME']==0.001:
                biases.append(im)
        print(len(biases),'images found in',biasdir)
        biases.sort()
        for n in range(len(biases)):
            print(str(n)+':',biases[n])
        header = fits.getheader(biases[0])
        medframe = mediancombine(biases)
        normmedframe = medframe/np.median(medframe)
        fits.writeto('NormMasterBias.fits', normmedframe,header,overwrite=True)
        print('NormMasterBias.fits written in',biasdir)
    except IndexError:
        print('No biases found. Moving to next night')

0 images found in E:/20190112/
No biases found. Moving to next night
0 images found in E:/20190113/
No biases found. Moving to next night
0 images found in E:/20190114/
No biases found. Moving to next night
0 images found in E:/20190115/
No biases found. Moving to next night
11 images found in E:/20190116/
0: c8499t0047b00.fits
1: c8499t0048b00.fits
2: c8499t0049b00.fits
3: c8499t0050b00.fits
4: c8499t0051b00.fits
5: c8499t0052b00.fits
6: c8499t0053b00.fits
7: c8499t0054b00.fits
8: c8499t0055b00.fits
9: c8499t0056b00.fits
10: c8499t0057b00.fits
NormMasterBias.fits written in E:/20190116/
11 images found in E:/20190117/
0: c8500t0287b00.fits
1: c8500t0288b00.fits
2: c8500t0289b00.fits
3: c8500t0290b00.fits
4: c8500t0291b00.fits
5: c8500t0292b00.fits
6: c8500t0293b00.fits
7: c8500t0294b00.fits
8: c8500t0295b00.fits
9: c8500t0296b00.fits
10: c8500t0297b00.fits
NormMasterBias.fits written in E:/20190117/
11 images found in E:/20190118/
0: c8500t0325b00.fits
1: c8500t0326b00.fits
2: c8500t0

In [ ]:
os.chdir('E:/20190122')
masterbias = fits.getdata('E:/20190121/NormMasterBias.fits')
allims = glob.glob('c*.fits')
Haflats = []
try:
    for im in allims:
        if fits.getheader(im)['FILTER2 ']=='200' and fits.getheader(im)['DECSTRNG']=='+13:00:03':
            Haflats.append(im)
    print(len(Haflats),'images found.')
    Haflats.sort()
    CustomBias(Haflats,masterbias)

except IndexError:
    print('No Ha flats found. Moving to next night')

In [ ]:
b_ims = glob.glob('b_*.fits')
Haflat=norm_combine_flats(b_ims)
header = fits.getheader(b_ims[0])
fits.writeto('HaFlat.fits',Haflat,header,overwrite=True)
print('HaFlat.fits has been written')
FileMover(b_ims,'b_ims')

In [ ]:
os.chdir('E:/20190122')
masterbias = fits.getdata('E:/20190121/NormMasterBias.fits')
allims = glob.glob('c*.fits')
HaOffflats = []
try:
    for im in allims:
        if fits.getheader(im)['FILTER2 ']=='202' and fits.getheader(im)['DECSTRNG']=='+13:00:03':
            HaOffflats.append(im)
    print(len(HaOffflats),'images found.')
    HaOffflats.sort()
    CustomBias(HaOffflats,masterbias)

except IndexError:
    print('No Ha flats found. Moving to next night')

In [ ]:
b_ims = glob.glob('b_*.fits')
HaOffflat=norm_combine_flats(b_ims)
header = fits.getheader(b_ims[0])
fits.writeto('HaOffFlat.fits',HaOffflat,header,overwrite=True)
print('HaOffFlat.fits has been written')
FileMover(b_ims,'b_ims')